In [1]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
from scipy.optimize import least_squares, curve_fit
from matplotlib import pylab as plt
import pandas as pd
import glob
import time
from tqdm.notebook import tqdm
import os
import random

import warnings
from scipy.optimize import OptimizeWarning
warnings.simplefilter("error", OptimizeWarning)

import seaborn as sns
sns.set()

In [2]:
filels = glob.glob("centered_csv_data/*.csv")

In [3]:
def fill_centered_df(df,time_steps = 200):
    df = pd.read_csv(file)
    df["mjd"] = df["mjd"] - df["mjd"][0]
    df["mjd"] = df["mjd"].round(decimals=0).astype('int')
    df = df.sort_values("fluxerr").groupby(["mjd", "band"]).first().reset_index()
    
    # df = df.drop(['detected_bool', 'ratio','ratio2bool'],1)
    df = df.drop(['detected_bool', 'ratio','ratio2bool','fluxerr'],1)

    df = pd.pivot_table(df, index=["mjd"], columns=["band"])
    df.columns= [f"{tup[0]}_{tup[1]}" for tup in df.columns]
    
    df = df.reindex(np.arange(0, time_steps))
    
    # rearranged_cols = [[f"flux_{f}",f"fluxerr_{f}"] for f in "ugrizY"]
    # rearranged_cols = [item for sublist in rearranged_cols for item in sublist]

    # rearranged_cols = [f"flux_{f}" for f in "ugrizY"] + [f"fluxerr_{f}" for f in "ugrizY"]
    rearranged_cols = [f"flux_{f}" for f in "ugrizY"]


    df = df[rearranged_cols]
    
    return df

In [4]:
if not os.path.isdir("preprocessed_csv_data"):
    os.mkdir("preprocessed_csv_data")

In [5]:
for file in tqdm(filels):
    df = pd.read_csv(file)
    df = fill_centered_df(df)
    df.to_csv(f"preprocessed_csv_data/{file.split('/')[-1]}",index=False)

  0%|          | 0/4000 [00:00<?, ?it/s]